<a href="https://colab.research.google.com/github/hypro2/step-by-step-colab/blob/main/bert_lora_classification_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hate speech detection
- model: kcbert-large-fintune-nsmc

In [21]:
!pip install -q evaluate
!pip install -q -U transformers peft accelerate optimum
!pip install --quiet bitsandbytes
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [3]:
import json
from datasets import Dataset
from peft import LoraConfig, TaskType
from transformers import BertForSequenceClassification, AutoTokenizer
import torch

from peft import get_peft_model

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import evaluate


import torch
import gc
gc.collect()
torch.cuda.empty_cache()

def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        data = [json.loads(line) for line in lines]
    return data

train_data = load_jsonl('./data/nikluge-au-2022-train.jsonl')
test_data = load_jsonl('./data/nikluge-au-2022-dev.jsonl')

# {"id": "nikluge-au-2022-train-000001", "input": "보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..", "output": 1}

train_dataset = Dataset.from_dict({
    'id': [item['id'] for item in train_data],
    'text': [item['input'] for item in train_data],
    'label': [item['output'] for item in train_data]
})

test_dataset = Dataset.from_dict({
    'id': [item['id'] for item in test_data],
    'text': [item['input'] for item in test_data],
    'label': [item['output'] for item in test_data]
})


# 기존에는.. model_id = "beomi/kcbert-base"
# path
model_id = "metamath/kcbert-large-finetuned-nsmc" # 2
# model_id = "Copycats/koelectra-base-v3-generalized-sentiment-analysis" # 3
# model_id = "jh0802/Korean-Hate-KCBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)


lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=8, lora_alpha=16, lora_dropout=0.1
)


model = BertForSequenceClassification.from_pretrained(
    model_id,
    num_labels=2,
)

model = get_peft_model(model, lora_config)


metric = evaluate.load("accuracy")

def compute_metrics(pred):
    labels = pred.label_ids
    logits = pred.predictions

    # Logits에서 레이블 0이 아닌 경우를 혐오 발언으로 간주
    preds = np.argmax(logits, axis=1)
    binary_preds = (preds != 0).astype(int)
    binary_labels = (labels != 0).astype(int)
    
    accuracy = accuracy_score(binary_labels, binary_preds)
    precision = precision_score(binary_labels, binary_preds)
    recall = recall_score(binary_labels, binary_preds)
    f1 = f1_score(binary_labels, binary_preds)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }
    
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  num_train_epochs=10,
                                  weight_decay=0.01,
                                #   learning_rate=2e-5,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1",
                                  greater_is_better=True,
                                  )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

# nikluge-iau-2023-test.jsonl를 가져와서 output을 예측하고, nikluge-iau-2023-test-predictions.jsonl로 저장
test_data = load_jsonl('./data/nikluge-au-2022-test.jsonl')
test_dataset = Dataset.from_dict({
    'id': [item['id'] for item in test_data],
    'text': [item['input'] for item in test_data],
})

tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)
predictions = trainer.predict(tokenized_test_dataset)

output = []
for idx, pred in enumerate(predictions.predictions):
    output.append({
        'id': test_data[idx]['id'],
        'input': test_data[idx]['input'],
        'output': 1 if pred.argmax() == 1 else 0
    })
    
# JSONL 파일로 저장
with open('./data/hate-detection-kcbert-large-nsmc.jsonl', 'w', encoding='utf-8') as file:
    for item in output:
        file.write(json.dumps(item, ensure_ascii=False) + '\n')

!head -n 5 ./data/nikluge-iau-2023-test-predictions.jsonl

100%|██████████| 3/3 [00:00<00:00, 19.33ba/s]
/home/link/anaconda3/envs/18th/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.275800,0.259506,0.914616,0.856688,0.950530,0.901173
2,0.233500,0.215111,0.930535,0.944515,0.882214,0.912302
3,0.219500,0.219216,0.927641,0.882804,0.949352,0.914869
4,0.210900,0.196520,0.935842,0.917249,0.926973,0.922086
5,0.192700,0.211225,0.936807,0.907955,0.941107,0.924234
6,0.191200,0.219274,0.931018,0.893096,0.944641,0.918145
7,0.171000,0.212810,0.934877,0.905682,0.938751,0.921920
8,0.171300,0.226032,0.934395,0.901917,0.942285,0.921659


/home/link/anaconda3/envs/18th/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/link/anaconda3/envs/18th/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/link/anaconda3/envs/18th/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/link/anaconda3/envs/18th/lib/python3.11/site-packages/huggingface_hub/file_downloa

{"id": "nikluge-au-2022-test-000001", "input": "극좌는 이 비겁자층을 제대로 요리할 줄 안다...", "output": 1}
{"id": "nikluge-au-2022-test-000002", "input": "내가 진짜 올 해 안에 차 산다!", "output": 0}
{"id": "nikluge-au-2022-test-000003", "input": "선거 때마다 불장난 하는 못된 버릇 대대손손 배워가지고 그러고 까불어대면, 너 나중에 뒷덜미에 혹난다???", "output": 1}
{"id": "nikluge-au-2022-test-000004", "input": "난 99년도에 이미 세상은 망해서 선한자들은 이미 모두 하늘로 올라갔고, 남은 우리가 살고 있는 지구는 생지옥이라는 생각을 가끔한다.", "output": 0}
{"id": "nikluge-au-2022-test-000005", "input": "피의자로 가는 싸가지 없는 쓰래기!", "output": 1}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [4]:
# nikluge-iau-2023-test.jsonl를 가져와서 output을 예측하고, nikluge-iau-2023-test-predictions.jsonl로 저장
test_data = load_jsonl('./data/nikluge-au-2022-test.jsonl')
test_dataset = Dataset.from_dict({
    'id': [item['id'] for item in test_data],
    'text': [item['input'] for item in test_data],
})

tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)
predictions = trainer.predict(tokenized_test_dataset)

output = []
for idx, pred in enumerate(predictions.predictions):
    output.append({
        'id': test_data[idx]['id'],
        'input': test_data[idx]['input'],
        'output': 1 if pred.argmax() == 1 else 0
    })
    
# JSONL 파일로 저장
with open('./data/hate-detection-kcbert-large-finetuned-nsmc-fin.jsonl', 'w', encoding='utf-8') as file:
    for item in output:
        file.write(json.dumps(item, ensure_ascii=False) + '\n')

!head -n 5 ./data/nikluge-iau-2023-test-predictions.jsonl

100%|██████████| 3/3 [00:00<00:00, 18.44ba/s]


{"id": "nikluge-au-2022-test-000001", "input": "극좌는 이 비겁자층을 제대로 요리할 줄 안다...", "output": 1}
{"id": "nikluge-au-2022-test-000002", "input": "내가 진짜 올 해 안에 차 산다!", "output": 0}
{"id": "nikluge-au-2022-test-000003", "input": "선거 때마다 불장난 하는 못된 버릇 대대손손 배워가지고 그러고 까불어대면, 너 나중에 뒷덜미에 혹난다???", "output": 1}
{"id": "nikluge-au-2022-test-000004", "input": "난 99년도에 이미 세상은 망해서 선한자들은 이미 모두 하늘로 올라갔고, 남은 우리가 살고 있는 지구는 생지옥이라는 생각을 가끔한다.", "output": 0}
{"id": "nikluge-au-2022-test-000005", "input": "피의자로 가는 싸가지 없는 쓰래기!", "output": 1}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
